In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import layers

In [2]:
train = pd.read_csv('train_data.csv')
train_label = pd.read_csv('train_label.csv')
test = pd.read_csv('test_data.csv')

In [3]:
train = train.drop('ID',axis=1)
train = np.array(train)
train = train.reshape((8960,131,3))

In [4]:
label = train_label.drop('ID', axis=1)
label = pd.get_dummies(label['type'])
label = np.array(label)

In [9]:
test = test.drop('ID', axis=1)
test = np.array(test)
test = test.reshape(2240,131,3)

In [5]:
rnn = Sequential([
    layers.LSTM(16, input_shape=(131,3), return_sequences=True),
    layers.LSTM(16, return_sequences=True),
    layers.LSTM(16, return_sequences=True),
    layers.LSTM(16, return_sequences=False),
    layers.Dense(4, activation='softmax')
  ])

In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
rnn.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=optimizer, metrics=['accuracy'])

callbacklist =[tf.keras.callbacks.ModelCheckpoint(filepath = 'LSTM.h5', moniter = 'val_accuracy', save_best_only=True)]

In [7]:
EPOCHS = 100
history = rnn.fit(train, label, callbacks=callbacklist,
                  epochs=EPOCHS, validation_split = 0.1, verbose=1)

Epoch 1/100
252/252 [==============================] - 10s 24ms/step - loss: 1.1091 - accuracy: 0.4032 - val_loss: 0.7209 - val_accuracy: 0.5324
Epoch 2/100
252/252 [==============================] - 4s 17ms/step - loss: 0.7258 - accuracy: 0.5159 - val_loss: 0.6833 - val_accuracy: 0.5636
Epoch 3/100
252/252 [==============================] - 4s 17ms/step - loss: 0.6809 - accuracy: 0.5892 - val_loss: 0.5925 - val_accuracy: 0.7522
Epoch 4/100
252/252 [==============================] - 4s 17ms/step - loss: 0.5268 - accuracy: 0.7568 - val_loss: 0.2976 - val_accuracy: 0.9208
Epoch 5/100
252/252 [==============================] - 4s 17ms/step - loss: 0.3968 - accuracy: 0.8462 - val_loss: 0.4197 - val_accuracy: 0.7812
Epoch 6/100
252/252 [==============================] - 4s 17ms/step - loss: 0.2871 - accuracy: 0.9002 - val_loss: 0.1626 - val_accuracy: 0.9710
Epoch 7/100
252/252 [==============================] - 4s 17ms/step - loss: 0.1636 - accuracy: 0.9605 - val_loss: 0.0955 - val_accuracy

252/252 [==============================] - 4s 17ms/step - loss: 0.0034 - accuracy: 0.9996 - val_loss: 0.0088 - val_accuracy: 0.9967
Epoch 58/100
252/252 [==============================] - 4s 17ms/step - loss: 6.9788e-04 - accuracy: 0.9998 - val_loss: 0.0084 - val_accuracy: 0.9978
Epoch 59/100
252/252 [==============================] - 4s 17ms/step - loss: 4.4007e-04 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 0.9978
Epoch 60/100
252/252 [==============================] - 4s 17ms/step - loss: 4.1358e-04 - accuracy: 1.0000 - val_loss: 0.0085 - val_accuracy: 0.9978
Epoch 61/100
252/252 [==============================] - 4s 17ms/step - loss: 3.3411e-04 - accuracy: 1.0000 - val_loss: 0.0086 - val_accuracy: 0.9978
Epoch 62/100
252/252 [==============================] - 4s 17ms/step - loss: 3.0142e-04 - accuracy: 1.0000 - val_loss: 0.0088 - val_accuracy: 0.9978
Epoch 63/100
252/252 [==============================] - 4s 17ms/step - loss: 2.6418e-04 - accuracy: 1.0000 - val_loss: 0.00

In [8]:
model = tf.keras.models.load_model('LSTM.h5')

In [10]:
pred = model.predict(test)
prediction = pred.argmax(axis=1)
prediction = prediction + 1

In [11]:
submit = pd.read_csv('answer_sample.csv')
submit['type'] = prediction
submit.to_csv('LSTM_answer.csv', index=False)